# Dockerfile VOLUME

_Dockerfile_ provides an instruction called `VOLUME` to allow users to configure storage volumes for their image to persist data in between container runs. The [_VOLUME_](https://docs.docker.com/develop/develop-images/dockerfile_best-practices/#volume) instruction create a new docker volume and mounts it onto your image at a specific directory. The image (container) can use this directory to store files which persist in between _docker runs_.

Take a look at _data/ex5/main.py_. We've enhanced our script to use a sqlite3 database to store the commands that have previously ran through our script. This is to give our script history; very similar to the bash terminal `history` command. The script has a new command line argument called `-d` which specifies the location of our sqlite3 database to store the historical commands.

In to save the command history, we've added a new class called `HistoryStore` with the following members:

- `init_db()`: creates a sqlite3 `history` table with 3 columns: *id,cmd,exec_time*
- `insert()`: inserts a row into history tables. cmd = list of command line args, exec_time = now
- `history()`: returns the last N rows from the history file in ascending order by id
- `stat()`: computes some stats of the history table: num of rows and last exec_time

Start a fresh directory and copy over our source files:

In [ ]:
mkdir docker/my-ex5
cp -r docker/ex5/* docker/my-ex5/

Modify the _Dockerfile_ to include:

In [ ]:
FROM python:3.7.10-slim-buster

# adding env variables
ENV INPUT_FILE="/home/deb-airports.csv" \
    NUM_ROWS=10

# set the working directory
WORKDIR /home

# add source files to the image
COPY * /home/

# install pip requirements
RUN pip3 install -r requirements.txt

# create a volume and initialize the history sqlite3 database
RUN mkdir /data
# initialize the sqlite database by calling the very first command: version
RUN python main.py -d /data/history.db version
# add the volume
# this will persist the /data directory as a volume going forward
VOLUME [ "/data" ]

# add an ENTRYPOINT
ENTRYPOINT ["python", "main.py", "-d", "/data/history.db"]
CMD ["print"]

**NOTE**:

1. We first create a directory called _/data_ and run our script once to initialize it with the very first history command
1. This will create a sqlite3 database under _/data/history.db_ and the version command into its history table
1. We then use the `VOLUME` command to persist this directory as a docker volume
1. The `VOLUME [ "/data" ]` command will persist the content of our _/data_ folder in between container runs

Build and run the image and observer the history behavior:

In [ ]:
cd docker/my-ex5

# build your image
docker build --rm -f Dockerfile -t dsa-deb-docker:ex5 .

# create a volume
docker volume create history-db


# run the container a few times
# the history is saved in between runs
docker run --rm -v history-db:/data dsa-deb-docker:ex5 history
docker run --rm -v history-db:/data dsa-deb-docker:ex5 print -n 25 -fs NY
docker run --rm -v history-db:/data dsa-deb-docker:ex5 print -n 10 -fs OR
docker run --rm -v history-db:/data dsa-deb-docker:ex5 print -n 30 -fs CA
docker run --rm -v history-db:/data dsa-deb-docker:ex5 history

**NOTE:**

Even though you specified a _VOLUME_ command in your Dockerfile, you still need to create a docker volume and pass it to your docker run! YES, it's stupid 😟️